In [117]:
import pickle
from pathlib import Path

import kagglehub
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB

In [102]:
# Download latest version
path = Path(kagglehub.dataset_download("kaggle/recipe-ingredients-dataset"))

print("Path to dataset files:", path)

Path to dataset files: /home/nates/.cache/kagglehub/datasets/kaggle/recipe-ingredients-dataset/versions/1


In [103]:
train_df = pd.read_json(path / "train.json")
test_df = pd.read_json(path / "test.json")

In [104]:
train_df.dtypes

id              int64
cuisine        object
ingredients    object
dtype: object

In [105]:
train_df = train_df.astype({"cuisine": "category"})

In [106]:
train_df['ingredients'] = train_df['ingredients'].apply(lambda x:  ' '.join(x))
test_df['ingredients'] = test_df['ingredients'].apply(lambda x:  ' '.join(x))

In [107]:
X, y = train_df["ingredients"], train_df["cuisine"]
X_test = test_df["ingredients"]

In [108]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [109]:
vectorizer = CountVectorizer()

X_train_bow = vectorizer.fit_transform(X_train).toarray()
X_val_bow = vectorizer.transform(X_val).toarray()
X_test_bow = vectorizer.transform(X_test).toarray()

In [110]:
grid_params = {

}

model = GridSearchCV(LogisticRegression(), grid_params, verbose=True, refit=True)

model.fit(X_train_bow,y_train)

y_pred = model.predict(X_val_bow)

accuracy_score(y_val, y_pred)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/nates/repos/recipe-recommender/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/nates/repos/recipe-recommender/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.7816467630421119

In [111]:
confusion_matrix(y_val, y_pred)

array([[  51,    1,    1,    0,    3,    3,    0,    1,    1,    2,    0,
           0,    0,   11,    0,    1,    7,    0,    2,    0],
       [   0,   71,    3,    0,    0,   15,    0,    2,   14,   15,    1,
           2,    0,    1,    0,    6,   26,    1,    0,    0],
       [   1,    2,  211,    2,    0,    6,    1,    2,    1,   18,    2,
           0,    0,    6,    0,    7,   65,    4,    0,    0],
       [   0,    2,    2,  427,    7,    3,    0,    3,    0,    4,    0,
          19,   14,    5,    0,    0,    4,    1,    7,   12],
       [   3,    1,    1,   20,   77,    4,    0,    0,    1,    6,    0,
           5,    2,    4,    0,    0,    6,    2,    2,    2],
       [   1,   11,    6,    2,    0,  354,    6,    2,   12,   90,    1,
           0,    0,   10,    0,   10,   40,    5,    0,    0],
       [   0,    1,    0,    0,    0,   11,  177,    2,    0,   44,    0,
           0,    1,    2,    3,    1,    2,    5,    0,    0],
       [   0,    1,    1,    2,    3,    

In [112]:
y_pred = model.predict(X_test_bow)

In [113]:
test_df["cuisine"] = y_pred

In [114]:
test_df

,id,ingredients,cuisine
0,18009,baking powder eggs all-purpose flour raisins m...,british
1,28583,sugar egg yolks corn starch cream of tartar ba...,southern_us
2,41580,sausage links fennel bulb fronds olive oil cub...,italian
3,29752,meat cuts file powder smoked sausage okra shri...,cajun_creole
4,35687,ground black pepper salt sausage casings leeks...,italian
...,...,...,...
9939,30246,large egg yolks fresh lemon juice sugar bourbo...,french
9940,36028,hot sauce butter sweet potatoes adobo sauce salt,southern_us
9941,22339,black pepper salt parmigiano reggiano cheese r...,italian
9942,42525,cheddar cheese cayenne paprika plum tomatoes g...,southern_us


In [115]:
test_df[["id", "cuisine"]].set_index("id").to_csv("submission.csv")

In [119]:
# save
with open('models/cuisine_model.pkl','wb') as f:
    pickle.dump(model,f)

# load
with open('models/cuisine_model.pkl', 'rb') as f:
    model = pickle.load(f)